# Surviving Titanic

## Introduction and initial question

Was there any factors that made it more possible to survive the Titanic shipwreck? Did it matter which class you were in or if you were a man or woman? 

## The dataset

Taken from Kaggle's competition, https://www.kaggle.com/c/titanic/data

### Data Dictionary

|Variable|Definition|Key|
|--------|----------|---|
|survival|Survival|0 = No, 1 = Yes|
|pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd|
|sex|Sex||
|Age|Age in years||	
|sibsp|# of siblings / spouses aboard the Titanic||
|parch|# of parents / children aboard the Titanic||
|ticket|Ticket number||
|fare|Passenger fare||
|cabin|Cabin number||
|embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampton|

### Variable Notes

* pclass: A proxy for socio-economic status (SES)
  * 1st = Upper
  * 2nd = Middle
  * 3rd = Lower

* age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

* sibsp: The dataset defines family relations in this way...
  * Sibling = brother, sister, stepbrother, stepsister
  * Spouse = husband, wife (mistresses and fiancés were ignored)

* parch: The dataset defines family relations in this way...
  * Parent = mother, father
  * Child = daughter, son, stepdaughter, stepson
*Some children travelled only with a nanny, therefore parch=0 for them.*

### Start by Importing necessary libraries

***Explain about the libraries used for this post***

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt
import seaborn

%matplotlib inline

### Now read in the data

Next we start by reading in the data using the pandas CSV reader.

In [ ]:
passengers = pd.read_csv('titanic-data.csv')
passengers.head()

In [ ]:
# Check info about the data we read
passengers.info()

We can see that Age, Cabin, and Embarked columns are missing values.
We either need to estimate the missing values or drop them. We can estimate missing ages but for our analysis I believe the non-null 714 values are enough. We choose to drop the rows with null values.

In [ ]:
passengers = passengers.dropna(subset=['Age'])

Cabin has to many missing values and according to wikipedia also a bias towards first class passengers. This column will be dropped during this analysis. Something that would be interesting to look up if cabin position would influence survival but this will be out of scope for this time.

In [ ]:
passengers = passengers.drop('Cabin', axis=1)

Embarked only had three missing vaules initially. However, it is hard to imagine a scenario where your point of origin would affect our survival rate. I choose to drop this column as well. 

In [ ]:
passengers = passengers.drop('Embarked', axis=1)

Further, we have some columns containing data that might be interesting. (Titles in Name, Ticket price in Fare, etc.) With more analysis these might contain valuable information but to keep our analysis simple, I will drop these as well.

In [ ]:
passengers = passengers.drop('Name', axis=1)
passengers = passengers.drop('PassengerId', axis=1)
passengers = passengers.drop('Fare', axis=1)

To make some plotting easier we add a column 'AgeGroup' where we split our sample passengers ages into four groups with equal spans.  
Also, upon further inspection it seems like SibSp/Parch and ticket groups by ticket numbers don't have any correlation making it hard to pinpoint which passenger are related to which etc. There might be a bit too much work to little return for analysing this data so we drop the two columns.

In [ ]:
passengers = passengers.drop('Ticket', axis=1)
passengers = passengers.drop('SibSp', axis=1)
passengers = passengers.drop('Parch', axis=1)

Lastly, to make column names more coherent we rename "Pclass" to just "Class" and to better described to contents we convert the Sex, Class, and Survived column's data types from Strings to Categories, and Integers to booleans respectively. 

In [ ]:
passengers = passengers.rename(columns={'Pclass': 'Class'})

passengers.Sex = passengers.Sex.astype('category')
passengers.Class = passengers.Class.astype('category')

passengers.Survived = passengers.Survived.apply(bool)

This gives us something like the below to work with.

In [ ]:
passengers.head()

### My own analysis of the variables types

***NEED TO WRITE***

### Descriptive stats

In [ ]:
passengers.Age.describe()

In [ ]:
passengers.Sex.describe()

In [ ]:
passengers.Class.describe()

In [ ]:
passengers.Survived.describe()

We can see here that we our sample mostly consists of third class travelers and a typical passenger were around 30 years old. Even after trimming of some rows we still have 714 entries in our sample to work with.

Let us also plot some different relations between class, sex, and age for the survivors and non-survivors for our data to see if we can find some interesting anomalities.

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))

survived_by_class = passengers.groupby(['Survived','Class']).size().unstack()
survived_by_class.plot(kind='bar', stacked=True, ax=axs[0])
axs[0].set_title('1. Survivors by Class')

survived_by_sex = passengers.groupby(['Survived', 'Sex']).size().unstack()
survived_by_sex.plot(kind='bar', stacked=True, ax=axs[1])
axs[1].set_title('2. Survivors by Sex')

plt.show()

From the above plots we can see some interesting facts. For example, there were proportionaly more women than men that survived the accident. This could be somewhat influenced by the fact that there where much more men than women in third class which also had higher rate of casualities. 

We also tried plotting the survival rate for passengers born the same year but too much noise did make it hard to get anything out of the plot. By bining the ages to intervals of 4 we can keep some of the noise down by increasing the number of samples per data point. From the second graph it looks light we might have a weak relationship between higher age and increased mortality rate.

In [ ]:
#Grouped by age in full years
survival_by_age = passengers.groupby(lambda x: int(passengers.loc[x].Age)).Survived

survival_rate_by_age = survival_by_age.apply(lambda x: x[x == True].count()/(x.count() * 1.0))

plt.scatter(survival_rate_by_age.index, survival_rate_by_age);

In [ ]:
#Group by age in intervals of 4
survival_by_age_group = passengers.groupby(lambda x: int(passengers.loc[x].Age/4)*4).Survived

In [ ]:
survival_rate_by_age_group = survival_by_age_group.apply(lambda x: x[x == True].count()/(x.count() * 1.0))

plt.scatter(survival_rate_by_age_group.index, survival_rate_by_age_group);

### Chi^2 test for relevance

After looking at the above three variables and plots we can hypothesize about there being three factors that all had some effect on the chance of survival.

1. Third class passengers where less likely to survive than 1 and 2 class passengers.
2. Women had a higher survivability than men.
3. Higher age decreased chance of survival.

I believe we can draw a conclusion already about what way the trends are headed for the above three variables but in order to test if what we are seeing is not just a coincidence from the values we have sampled we will perform Chi^2 test on each of them to see if there are any statistically significant change depending on these variables.

In [ ]:
# First we calculate the total survival rate

survival_rate = passengers[passengers.Survived == True].Survived.count()/(passengers.Survived.count()*1.0)
survival_rate

In [ ]:
# Starting with the Chi^2 for survivors by class.
survivors_by_class = pd.DataFrame(passengers.groupby(['Class', 'Survived']).size().unstack()[True])
survivors_by_class = survivors_by_class.rename(columns={True: 'Survivors'})

survivors_by_class['Total'] = passengers.groupby(['Class']).size()
survivors_by_class['Expected'] = survivors_by_class['Total'] * survival_rate
survivors_by_class['Difference'] = survivors_by_class['Survivors'] - survivors_by_class['Expected'] 
survivors_by_class['Difference2'] = survivors_by_class['Difference']**2
survivors_by_class['Proportion'] = survivors_by_class['Difference2'] / survivors_by_class['Expected']

survivors_by_class

In [ ]:
survivors_by_class['Proportion'].sum()

Since df's definition is (rows - 1)(columns - 1) we get df = 2
And we will use an alpha of 0.05.

Looking up the CHI^2 value in a table gives us the critical value: 5.991

This means there are a dependency between class and survival rate with a probability of 95%.

In [ ]:
# Next up is the chi square test for sex.
survivors_by_sex = pd.DataFrame(
    passengers.groupby(['Sex', 'Survived']).size().unstack()[True])
survivors_by_sex = survivors_by_sex.rename(
    columns={True: 'Survivors'})

survivors_by_sex['Total'] = passengers.groupby(['Sex']).size()
survivors_by_sex['Expected'] = survivors_by_sex['Total'] * survival_rate
survivors_by_sex['Difference'] = survivors_by_sex['Survivors'] - survivors_by_sex['Expected'] 
survivors_by_sex['Difference2'] = survivors_by_sex['Difference']**2
survivors_by_sex['Proportion'] = survivors_by_sex['Difference2'] / survivors_by_sex['Expected']

survivors_by_sex

In [ ]:
survivors_by_sex['Proportion'].sum()

Since df's definition is (rows - 1)(columns - 1) we get df = 1
And we will use an alpha of 0.05.

Looking up the CHI^2 value in a table gives us the critical value: 3.841

This means there are a dependency between sex and survival rate with a probability of 95%.

### Logistical regression for probabilities

Next we will investigate the relation between Age and survival with logistical regression from the statsmodels package.

In [ ]:
import statsmodels.api as sm

# Start by dropping class and sex, for now...
passenger_ages = passengers.drop(['Class', 'Sex'], axis=1)


In [ ]:
logit = sm.Logit(passenger_ages.Survived, passenger_ages.Age)

# Fit the model
result = logit.fit()

result.summary()


As a final step we include variables for Class and Age to see how all three affects the survival rate.

In [ ]:
passenger_dummies = passenger_ages.join(
    pd.get_dummies(passengers['Class'], prefix='Class'))
passenger_dummies = passenger_dummies.join(
    pd.get_dummies(passengers['Sex']))

# To create a baseline, our typical passenger, 
# we drop the class 3 and male column.
passenger_dummies.drop('Class_3', axis=1, inplace=True)
passenger_dummies.drop('male', axis=1, inplace=True)

passenger_dummies.head()

In [ ]:
logit = sm.Logit(passenger_dummies['Survived'], 
                 passenger_dummies[passenger_dummies.columns[1:]])

# Fit the model
result = logit.fit()

result.summary()

In [ ]:
# Last step, visualizing the probability for all possible data points
ages = np.linspace(0, 80, 81, dtype=int)

# define cartesian function
def cartesian(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype
    
    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
        
    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

#enumerate all possible combinations of dummy variables (0=male, 1=female)
combinations = pd.DataFrame(cartesian([ages, [1, 2, 3], [0, 1]]))

#recreate dummies
combinations.columns = ['age', 'class', 'female']

dummy_classes = pd.get_dummies(combinations['class'])
dummy_classes.columns = ['class_1', 'class_2', 'class_3']

dummy_sexes = pd.get_dummies(combinations['female'])
dummy_sexes.columns = ['male', 'female']

#keep what we need for predictions and join with dummies

combinations = pd.DataFrame(combinations['age']).join(dummy_classes.ix[:, :'class_2'])
combinations = combinations.join(dummy_sexes.ix[:, 'female':])

combinations['survival_prediction'] = result.predict(combinations)

# Predicted survival probabilities for a average aged passenger.
combinations[180:186]

In [ ]:
#Pivoting table for plotting
pivot = pd.pivot_table(combinations, 
                       values=['survival_prediction'],
                       index=['age'],
                       columns=['female', 'class_1', 'class_2'])

pivot.head()

In [ ]:
pivot.plot();

### Conclusion